In [ ]:
# Merge features from various datasets, including creating new ones.

In [9]:
# Merge yelp business dataset with population & population density from simple maps dataset

import pandas as pd

yelpBus_df = pd.read_json('/Users/james/TIM147/yelp_dataset/yelp_academic_dataset_business.json',lines=True) # Import yelp business data
simple_df = pd.read_csv('/Users/james/Tim147/simplemaps_uszips_basicv1/uszips.csv',) # Import simple maps data
income_df = pd.read_csv('/Users/james/Tim147/income_dataset/kaggle_income.csv', encoding='iso-8859-1 ',)

In [3]:
yelpBus_df['postal_code'] = pd.to_numeric(yelpBus_df['postal_code'], errors='coerce') # Convert postal code from object to float64
yelpBus_df['postal_code'] = yelpBus_df['postal_code'].astype('Int64') # Convert postal code from float64 to Int64, with capital I so that it works with nan values

In [4]:
simple_df = simple_df[['zip','population','density']]
simple_df = simple_df.rename(columns = {'zip': 'postal_code', 'density': 'pop_density'})

In [6]:
merged_df = pd.merge(yelpBus_df, simple_df, on='postal_code', how='left') # Merge 

In [10]:
# Merge income on nearest latitude & longitude

import geopandas as gpd
from geopandas.tools import sjoin_nearest # Spatial join
from shapely.geometry import Point # 

income_df = income_df[['Lat','Lon','Median']]
income_df = income_df.rename(columns = {'Median': 'median_income'})

In [11]:
left_geo = [Point(xy) for xy in zip(merged_df['latitude'],merged_df['longitude'])]
right_geo = [Point(xy) for xy in zip(income_df['Lat'],income_df['Lon'])]

In [12]:
merged_gdf = gpd.GeoDataFrame(merged_df, geometry=left_geo)
income_gdf = gpd.GeoDataFrame(income_df, geometry=right_geo)

In [13]:
sjoined_gdf = sjoin_nearest(merged_gdf, income_gdf, how='left')

In [14]:
sjoined_df = sjoined_gdf.drop(['geometry','index_right','Lat','Lon'], axis=1)

In [18]:
sjoined_df.to_csv('yelp_business_plus.csv')